In [ ]:
from icecube import dataio, icetray , dataclasses 


In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)   # tüm kolonlar


In [ ]:
from collections.abc import Iterable

In [ ]:
GCD_PATH = "/project/6008051/pone_simulation/GCD_Library/PONE_800mGrid.i3.gz"
data_file = dataio.I3File(GCD_PATH)
geometry = data_file.pop_frame()


In [ ]:
# dir(geometry)

In [ ]:
print(type(geometry))
print("Stop:", geometry.Stop)   
print("Key's:", list(geometry.keys()))


In [ ]:
# Small helper: print the type and a short summary for each key in the frame
for key in geometry.keys():
    obj = geometry[key]
    print("="*60)
    print(f"Key: {key}")
    print("Type:", type(obj))
    try:
        print("dir(obj) first 10:", dir(obj)[:10])
    except Exception as e:
        print("dir(obj) could not be obtained:", e)

# - Geometry.Subdetectors

In [ ]:
subdets = geometry["Subdetectors"]
print(type(subdets))
print("Contents of Subdetectors:")
for name in subdets:
    print("  -", name)


In [ ]:
len(subdets)
# number of OM

In [ ]:
# print(subdets)
# (name of these subdetectors: Upgrade)

# - Geometry.StartTime and Geometry.EndTime
###### (no useful information here)

In [ ]:
start = geometry["StartTime"]
end   = geometry["EndTime"]
print(start)
print(end)

In [ ]:
start

In [ ]:
end

In [ ]:
print(repr(start))
print(str(start))


In [ ]:
print(repr(end))
print(str(end))


# - Geometry.I3OMGeoMap
##### it has position information, but it is on OM level, not PMT level: same position for all PMTs in one OM

In [ ]:
om_map = geometry["I3OMGeoMap"]
print(type(om_map))
print("Number of OMs:", len(om_map))

# 340 * 20 * 16


In [ ]:
om_map.keys()

In [ ]:
print("\nFirst 5 OMs:")
for i, (omkey, omgeo) in enumerate(om_map.items()):
    pos = omgeo.position
    print(f"OMKey={omkey}  x={pos.x:.2f}  y={pos.y:.2f}  z={pos.z:.2f}")
    if i >= 4:
        break


In [ ]:
# 1) I3OMGeoMap -> pandas DataFrame
rows = []
for omkey, omgeo in om_map.items():
    pos = omgeo.position
    rows.append({
        "string": omkey.string,
        "om": omkey.om,
        "pmt": getattr(omkey, "pmt", None),
        "x": pos.x,
        "y": pos.y,
        "z": pos.z,
    })

om_df = pd.DataFrame(rows)
om_df.head()


In [ ]:
om_df["pos_tuple"] = list(
    zip(om_df["x"].round(6), om_df["y"].round(6), om_df["z"].round(6))
)

uniq_per_om = om_df.groupby(["string", "om"])["pos_tuple"].nunique()

print("Max distinct positions per (string, om):", uniq_per_om.max())

if uniq_per_om.max() == 1:
    print("=> Positions are OM-level (same for all PMTs within an OM).")
else:
    print("=> Positions are PMT-level (they vary between PMTs in the same OM).")

example_key = uniq_per_om.idxmax()
print("\nExample (string, om):", example_key)
print(om_df[(om_df["string"] == example_key[0]) & (om_df["om"] == example_key[1])])


# - I3ModuleGeoMap

In [ ]:
mod_map = geometry["I3ModuleGeoMap"]
print(type(mod_map))
print("Number of modules:", len(mod_map))

# 340 * 20

In [ ]:
print(mod_map)

In [ ]:
first_key, first_geo = next(iter(mod_map.items()))

print("Example key:", first_key)
print("Type      :", type(first_geo))

print("\nPublic attributes of I3ModuleGeoMap:")
for a in dir(first_geo):
    if not a.startswith("_"):
        try:
            v = getattr(first_geo, a)
        except Exception:
            v = "<error reading>"
        print(f"  {a}: {v}")


In [ ]:
rows = []

for mkey, mgeo in mod_map.items():
    pos = mgeo.pos
    ori = mgeo.orientation
    dir_vec = mgeo.dir          # I3Direction

    rows.append({
        "string": mkey.string,
        "module": mkey.om,

        "x": pos.x,
        "y": pos.y,
        "z": pos.z,

        "dir_x": dir_vec.x,
        "dir_y": dir_vec.y,
        "dir_z": dir_vec.z,
        "dir_zenith": dir_vec.zenith,
        "dir_azimuth": dir_vec.azimuth,

        "up_x": ori.up.x,
        "up_y": ori.up.y,
        "up_z": ori.up.z,

        "right_x": ori.right.x,
        "right_y": ori.right.y,
        "right_z": ori.right.z,

        "module_type": str(mgeo.module_type),  # mDOM, PDOM, ...
        "radius": mgeo.radius,
        "rr": mgeo.rr,
        "rz": mgeo.rz,
        "height": mgeo.height,
    })

mod_df = pd.DataFrame(rows)
mod_df.head()


In [ ]:
mod_df.shape        


In [ ]:
unique_counts = mod_df.nunique(dropna=False)

constant_cols = unique_counts[unique_counts == 1].index.tolist()
varying_cols  = unique_counts[unique_counts > 1].index.tolist()

print("Columns that are constant across all rows:")
for c in constant_cols:
    print("  -", c)

print("\nColumns that vary:")
for c in varying_cols:
    print("  -", c)


In [ ]:
#### do all have the same orientation???
#### how about the orientation of PMTs

In [ ]:
# - I3Geometry
##### (no useful information here)

In [ ]:
geo = geometry["I3Geometry"]

print("Type:", type(geo))
print("Attributes (first 30):")
print(dir(geo))


In [ ]:
print(geo)

In [ ]:
# All attributes
all_attrs = dir(geo)

# Non-dunder attributes
public_attrs = [a for a in all_attrs if not a.startswith("_")]

print("Public attrs:")
for a in public_attrs:
    print("  -", a)


In [ ]:
rows = []
for a in public_attrs:
    value = getattr(geo, a)
    # try to get length if it behaves like a container
    try:
        n = len(value)
    except TypeError:
        n = None

    rows.append({
        "attr": a,
        "type": type(value).__name__,
        "len": n,
        "repr_sample": repr(value),  
    })

geo_attrs_df = pd.DataFrame(rows)
geo_attrs_df
